In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import warnings

# Load environment variables
_ = load_dotenv(find_dotenv())

# Append the submodules path to the local libs directory
repo_dir = Path().resolve()
sys.path.append(str(repo_dir / 'libs'))

# Ensure the symlink exists (assuming setup_symlink.py has been executed)
symlink_path = repo_dir / 'libs' / 'NLP_on_multilingual_coin_datasets'
if not symlink_path.exists():
    print(f"Error: Symlink {symlink_path} does not exist. Run setup_symlink.py first.")
    sys.exit(1)

# Import the custom modules after ensuring symlink is in place
from NLP_on_multilingual_coin_datasets.cnt.io import Database_Connection
from modules.loading_preprocessed_designs import PreprocessingConfig

# Set up pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

# Suppress warnings
warnings.filterwarnings('ignore')

prep_cfg = PreprocessingConfig


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
database = prep_cfg.database

connection_string = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{database}"
dc = Database_Connection(connection_string)

In [3]:
from sqlalchemy import create_engine

engine = create_engine(connection_string)
connection = engine.connect()
print("Connection established:", connection)
connection.close()

ModuleNotFoundError: No module named 'mysql'

In [4]:
RE_query = """select design_id, 
(select design_en_changed from nlp_training_designs as nlp where re.design_id=nlp.id) as design_en,
(select name_en from nlp_list_entities as ner where ner.id=re.subject) as s, 
(select class from nlp_list_entities as ner where ner.id=re.subject) as subject_class, 
(select name_en from nlp_list_entities as ner where ner.id=re.predicate) as p, 
(select name_en from nlp_list_entities as ner where ner.id=re.object) as o, 
(select class from nlp_list_entities as ner where ner.id=re.object) as object_class
from nlp_relation_extraction_en_v2 as re;
"""

df_RE_groundtruth= dc.create_own_query(RE_query)

print("Database connection type:", type(dc.mysql_connection))


if df_RE_groundtruth is None or df_RE_groundtruth.empty:
    print("No data returned from the query.")
else:
    print(df_RE_groundtruth.head())

# sort by design id ascending
df_RE_groundtruth = df_RE_groundtruth.sort_values(by=['design_id'])


df_RE_groundtruth.head(20)



SQL query failed.
Database connection type: <class 'str'>
No data returned from the query.


AttributeError: 'NoneType' object has no attribute 'sort_values'

In [ ]:
df_RE_groundtruth.to_json(prep_cfg.json_path + "/RE_groundtruth.json", orient="records")
df_spo_triples = pd.read_json(prep_cfg.json_path + "/subject_predicate_object_triples.json", orient="records")

In [ ]:
# filter df_spo for columns
columns = ['design_id', 's_o_id', 's', 'subject_class', 'predicate', 'o', 'object_class', 
           "validity_pred", "comment_pred", "implicit_pred", "design_en"
           ]

df_triples = df_spo_triples[columns].copy()
df_triples = df_triples.rename(columns={"predicate": "p"})

df_triples.head(5)

In [ ]:
df_triples.to_json(prep_cfg.json_path + "/RE_new_datachallenge.json", orient="records")

df_RE = pd.merge(df_triples, df_RE_groundtruth, on="design_id", how="inner")
df_RE.info()

In [ ]:
df_RE.head(5)

In [ ]:
df_RE_compare = df_RE.rename(columns={
    's_x': 's_new',
    'subject_class_x': 's_class_new',
    'p_x': 'p_new',
    'o_x': 'o_new',
    'object_class_x': 'o_class_new',
    's_y': 's_old',
    'subject_class_y': 's_class_old',
    'p_y': 'p_old',
    'o_y': 'o_old',
    'object_class_y': 'o_class_old',
    'design_en_x': 'design_en'
})[[
    'design_id', 's_o_id', 's_new', 's_class_new', 'p_new', 'o_new', 'o_class_new',
    's_old', 's_class_old', 'p_old', 'o_old', 'o_class_old',
    'design_en', 'validity_pred', 'comment_pred', 'implicit_pred'
]].copy()

In [ ]:
df_RE_compare.to_json(prep_cfg.json_path + "/RE_compare_groundtruth_vs_datachallenge.json", orient="records")
df_RE_compare.head(5)

# Comparison new triples vs Ground truth


- create json file in (exisiting) RE formatting
    - triples, keys and some values?
    - map verbs/predicates to classes

- Compare (some) triples with RE ground truth
    - Read RE ground truth
    - Check if some design_ids match
        - if yes compare them
        - if not, compute some
    - create one Df merge over all matching triples.



In [ ]:
df_spo_triples = pd.read_json(prep_cfg.json_path + "/subject_predicate_object_triples.json", orient="records")
df_spo_triples.info()

In [ ]:
# filter all predicate == "NULL" 

df_spo_triples = df_spo_triples[df_spo_triples["predicate"] != "NULL"]

df_spo_triples["validity_pred"].value_counts()

In [ ]:
one = 5845
zero = 706
minus = 390
overall = one + zero + minus

rel_one = round(one / overall, 2)
rel_zero = round(zero / overall, 2)
rel_minus = round(minus / overall, 2)

print(f"relative 1: {rel_one}, \nrelative 0: {rel_zero}, \nrelative -1: {rel_minus}")

In [ ]:
columns_new = ['design_id', 's', 'subject_class', 'predicate', 'o', 'object_class', 'validity_pred', 'implicit_pred', 'design_en']
df_triples_new = df_spo_triples[columns_new].copy()
df_triples_new = df_triples_new.rename(columns={"predicate": "p"})
df_triples_new.info()

In [ ]:
# Aggregation functions
def agg_long(x):
    return list(zip(x['s'], x['subject_class'], x['p'], x['o'], x['object_class'], x['validity_pred'], x['implicit_pred']))

def agg_short(x):
    return list(zip(x['s'], x['subject_class'], x['p'], x['o'], x['object_class']))

# Group by 'design_id' and aggregate
df_aggregated = df_triples_new.groupby(['design_id', 'design_en']).apply(lambda x: pd.Series({
    'l_spo_long': agg_long(x),
    'l_spo_short': agg_short(x)
})).reset_index()

df_aggregated.head(5)


In [ ]:
# load groundtruth data from json   
df_RE_groundtruth = pd.read_json(prep_cfg.json_path + "/RE_groundtruth.json", orient="records")
df_RE_groundtruth.info()

In [ ]:
# Group by 'design_id' and 'design_en' and aggregate
df_aggregated_groundtruth = df_RE_groundtruth.groupby(['design_id', 'design_en']).apply(lambda x: pd.Series({
    'l_spo_short': agg_short(x)
})).reset_index()

print(df_aggregated_groundtruth.info())
df_aggregated_groundtruth.drop(columns="design_en", inplace=True)
df_aggregated_groundtruth.head(20)

In [ ]:
# Merge DataFrames on 'design_id'
df_merged = pd.merge(df_aggregated, df_aggregated_groundtruth, on='design_id', suffixes=('_new', '_gt'))
print(df_merged.info())


In [ ]:
# Function to compare lists
def compare_lists(row):
    list_agg = row['l_spo_short_new']
    list_gt = row['l_spo_short_gt']
    
    set_agg = set(list_agg)
    set_gt = set(list_gt)
    
    if set_agg == set_gt:
        return 0
    
    agg_dict = {(s, o): p for s, _, p, o, _ in list_agg}
    gt_dict = {(s, o): p for s, _, p, o, _ in list_gt}
    
    if len(list_agg) == len(list_gt) and all((s, o) in agg_dict and agg_dict[(s, o)] != p for (s, o), p in gt_dict.items()):
        return 0.1
    
    if len(list_agg) > len(list_gt) and all((s, o) in agg_dict and agg_dict[(s, o)] != p for (s, o), p in gt_dict.items()):
        return 1.1
    
    if set_agg.issuperset(set_gt):
        return 1
    
    return -1

0.0: Exact match of l_spo_short lists in both DataFrames.
1.0: Aggregated list contains all elements of the ground truth list plus more.
-1.0: Aggregated list does not cover all elements of the ground truth list.
0.1: Lists are the same length; s and o match but p values differ.
1.1: Aggregated list is longer; s and o match but p values differ.

In [ ]:
df_merged['comparison_result'] = df_merged.apply(compare_lists, axis=1)
print(df_merged['comparison_result'].value_counts())
print(df_merged.info())  


In [ ]:
one = 262
zero = 186
minus = 301
overall = one + zero + minus

rel_one = round(one / overall, 2)
rel_zero = round(zero / overall, 2)
rel_minus = round(minus / overall, 2)

print(f"rel match ++: {rel_one}, \nrel match: {rel_zero}, \nrel diff: {rel_minus}")

In [ ]:
#save df_merged to json
df_merged.to_json(prep_cfg.json_path + "/RE_compare_groundtruth_vs_datachallenge_aggregated.json", orient="records")

In [ ]:
df_merged.info()


In [ ]:
df_merged